In [83]:
import numpy as np
import os

In [193]:
root = '../orange_demo/video/0134_c6_t0860to0945/'

In [194]:
# from collections import defaultdict
frame_order = {}
paths = []
pids = []
cnt = 0
with open(os.path.join(root,'img_path.txt'), 'r') as f:
    for path in f.readlines():
        p = path.strip()
        # print(p)
        paths.append(p)
        # print(p.strip('/'))
        # print(p.strip('/')[-1][:4])
        pid = int(p.split('/')[-1][:4])
        pids.append(pid)
        # 第一张为query
        if not cnt:
            cnt += 1
            continue
        cnt += 1
        time = p.split('/')[-1][-8:-4]
        if time not in frame_order:
            frame_order[time] = 1
        else:
            frame_order[time] += 1
        # break


# camids = []
# with open(os.path.join(root,'camids.txt'), 'r') as f:
#     for camid in f.readlines():
#         camids.append(int(camid.strip()))

# with open(os.path.join(root,'pids.txt'), 'r') as f:
#     for pid in f.readlines():
#         pids.append(int(pid.strip()))

num_query = 1
num_gallery = len(paths) - num_query

In [195]:
frame_order

{'0860': 25,
 '0865': 27,
 '0870': 29,
 '0875': 31,
 '0880': 30,
 '0885': 26,
 '0890': 26,
 '0895': 28,
 '0900': 30,
 '0905': 31,
 '0910': 33,
 '0915': 34,
 '0920': 34,
 '0925': 35,
 '0930': 36,
 '0935': 35,
 '0940': 34,
 '0945': 30}

In [196]:
num_gallery

554

In [197]:
dist = np.load(os.path.join(root, 'dist.npy'))

In [198]:
dist.shape

(1, 554)

In [199]:
dists = []
start = 0
end = 0
for _, v in frame_order.items():
    end += v
    cur_d = dist[:, start:end]
    # print(start, end, cur_d.shape)
    dists.append(cur_d)
    start = end

In [200]:
len(dists[0][0])

25

In [201]:
def sort_rows_and_get_indices(array):
    # 获取每一行排序后的索引
    # sorted_indices = np.argsort(array, axis=1)[:, ::-1]
    sorted_indices = np.argsort(array, axis=1)


    # 对每一行进行排序
    sorted_rows = np.take_along_axis(array, sorted_indices, axis=1)

    return sorted_rows, sorted_indices

# 举例一个二维数组
example_array = np.array([[3, 1, 4, 1],
                          [5, 9, 2, 6],
                          [5, 3, 5, 8]])

# # 对每一行进行排序并获取索引
# sorted_rows, indices = sort_rows_and_get_indices(example_array)

# # 打印结果
# print("原始数组:")
# print(example_array)
# print("\n每一行排序后的数组:")
# print(sorted_rows)
# print("\n排序后元素在原始行中的索引:")
# print(indices)


In [202]:
sorted_dists = []
sorted_indices = []
for d in dists:
    sorted_dist, sorted_indice = sort_rows_and_get_indices(d)
    sorted_dists.append(sorted_dist)
    sorted_indices.append(sorted_indice)

In [203]:
for s in sorted_dists:
    print(s[0][0])

0.6461003
0.69146246
5.364418e-07
0.5470813
0.6091796
0.5070686
0.7339752
0.6778549
0.67650115
0.6057316
0.59031594
0.667405
0.68005574
0.6060013
0.6299554
0.75499773
0.734384
0.7071471


In [204]:
sorted_indices[0].shape[1]

25

In [205]:
from tqdm import tqdm
def get_top_k_similar_pids(array, g_start=0, k=0):
    top_k_matches = []
    for q_id, g_ids in tqdm(enumerate(sorted_indice)):
        q_pid = pids[q_id]
        # q_camid = camids[q_id]
        l = 0
        for g_id in g_ids:
            if l >= k:
                break
            # g_camid = camids[num_query + g_id]
            # if q_camid == g_camid:
            #     continue
            g_pid = pids[num_query + g_start + g_id]
            tmp_list=paths[num_query + g_start + g_id], g_pid==q_pid
            l += 1
        top_k_matches.append(tmp_list)
    return top_k_matches

In [206]:
top_5_matches_list = []
g_start = 0
for sorted_indice in sorted_indices:
    top_5_matches = get_top_k_similar_pids(sorted_indice, g_start, 1)
    g_start += sorted_indice.shape[1]
    top_5_matches_list.append(top_5_matches)

1it [00:00, 8943.08it/s]
1it [00:00, 23967.45it/s]
1it [00:00, 24244.53it/s]
1it [00:00, 27235.74it/s]
1it [00:00, 27235.74it/s]
1it [00:00, 17697.49it/s]
1it [00:00, 17476.27it/s]
1it [00:00, 17403.75it/s]
1it [00:00, 17260.51it/s]
1it [00:00, 17549.39it/s]
1it [00:00, 16777.22it/s]
1it [00:00, 17403.75it/s]
1it [00:00, 17476.27it/s]
1it [00:00, 14873.42it/s]
1it [00:00, 16320.25it/s]
1it [00:00, 17119.61it/s]
1it [00:00, 18001.30it/s]
1it [00:00, 21959.71it/s]


In [207]:
top_5_matches_list

[[('datasets/Orange_demo_wild/baseline/gallery0134_c6/860/0134_c6_t0860.png',
   True)],
 [('datasets/Orange_demo_wild/baseline/gallery0134_c6/865/0134_c6_t0865.png',
   True)],
 [('datasets/Orange_demo_wild/baseline/gallery0134_c6/870/0134_c6_t0870.png',
   True)],
 [('datasets/Orange_demo_wild/baseline/gallery0134_c6/875/0134_c6_t0875.png',
   True)],
 [('datasets/Orange_demo_wild/baseline/gallery0134_c6/880/0134_c6_t0880.png',
   True)],
 [('datasets/Orange_demo_wild/baseline/gallery0134_c6/885/0134_c6_t0885.png',
   True)],
 [('datasets/Orange_demo_wild/baseline/gallery0134_c6/890/0134_c6_t0890.png',
   True)],
 [('datasets/Orange_demo_wild/baseline/gallery0134_c6/895/0134_c6_t0895.png',
   True)],
 [('datasets/Orange_demo_wild/baseline/gallery0134_c6/900/0134_c6_t0900.png',
   True)],
 [('datasets/Orange_demo_wild/baseline/gallery0134_c6/905/0134_c6_t0905.png',
   True)],
 [('datasets/Orange_demo_wild/baseline/gallery0134_c6/910/0134_c6_t0910.png',
   True)],
 [('datasets/Orange_d

In [208]:
trues = 0
cnt = 0
for id, item in enumerate(top_5_matches_list):
    # print(item)
    name, preci = item[0]
    if preci:
        trues += 1
    else:
        print(name, id)
    cnt += 1

datasets/Orange_demo_wild/baseline/gallery0134_c6/935/0137_c6_t0935.png 15
datasets/Orange_demo_wild/baseline/gallery0134_c6/940/0216_c6_t0940.png 16
datasets/Orange_demo_wild/baseline/gallery0134_c6/945/0194_c6_t0945.png 17


In [209]:
id = 0
s = 0
for k, v in frame_order.items():
    s += v
    print(id,k)
    id += 1
print(s)

0 0860
1 0865
2 0870
3 0875
4 0880
5 0885
6 0890
7 0895
8 0900
9 0905
10 0910
11 0915
12 0920
13 0925
14 0930
15 0935
16 0940
17 0945
554


In [210]:
with open(os.path.join(root, 'results.txt'), 'w') as f:
    for id, item in enumerate(top_5_matches_list):
        # print(item)
        name, preci = item[0]
        f.write(name)
        f.write('\t')
        f.write(str(preci))
        f.write('\t')
        f.write(str(sorted_dists[id][0][0]))
        f.write('\n')


for i, x in enumerate(sorted_indice):
    print(x[:10])
    for item in x:
        print(item)
        break
    break
    pass

In [22]:
# import json

In [23]:
# with open(os.path.join(root, 'matches.json'), 'w') as f:
#     json.dump(my_dict, f, indent=4, sort_keys=True)